In [1]:
import requests
import base64
import os

api_key = "927900cc-745e-4db9-9556-3fa7833a46ae"

def create_finetune(
    zip_path,
    finetune_comment="Example finetuning for character modeling",
    trigger_word="TOK",
    mode="character",
    iterations=100,
    learning_rate=0.00001,
    captioning=True,
    priority="quality",
    finetune_type="full",
    lora_rank=32,
):

    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"ZIP file not found at {zip_path}")

    assert mode in ["character", "product", "style", "general"]

    with open(zip_path, "rb") as file:
        encoded_zip = base64.b64encode(file.read()).decode("utf-8")

    url = "https://api.us1.bfl.ai/v1/finetune"
    headers = {
        "Content-Type": "application/json",
        "X-Key": api_key,
    }
    payload = {
        "finetune_comment": finetune_comment,
        "trigger_word": trigger_word,
        "file_data": encoded_zip,
        "iterations": iterations,
        "mode": mode,
        "learning_rate": learning_rate,
        "captioning": captioning,
        "priority": priority,
        "lora_rank": lora_rank,
        "finetune_type": finetune_type,
    }

    response = requests.post(url, headers=headers, json=payload)
    try:
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        raise requests.exceptions.RequestException(
            f"Finetune request failed:\n{str(e)}\n{response.content.decode()}"
        )
    


def add_finetune(finetune_id, finetune_name):
    url = "https://api.us1.bfl.ai/v1/my_finetunes"
    headers = {
        "Content-Type": "application/json",
        "X-Key": api_key,
    }

    response = requests.get(url, headers=headers)
    try:
        response.raise_for_status()
        available_finetunes = response.json()
    except requests.exceptions.RequestException as e:
        raise requests.exceptions.RequestException(
            f"Failed to fetch finetune status:\n{str(e)}\n{response.content.decode()}"
        )
    
    if finetune_id not in available_finetunes['finetunes']:
        raise ValueError(f"Finetune ID '{finetune_id}' is not available in your finetunes list.")
    
    try:
        with open("finetunes.py", "r") as f:
            content = f.read()
            dict_start = content.find("{")
            dict_end = content.rfind("}") + 1
            dict_content = content[dict_start:dict_end]
            FINETUNES = eval(dict_content)
    except FileNotFoundError:
        FINETUNES = {}
    except Exception as e:
        print(f"Ошибка при загрузке файла: {e}")
        FINETUNES = {}

    FINETUNES[finetune_name] = finetune_id

    with open("finetunes.py", "w") as f:
        f.write(f"FINETUNES = {FINETUNES}")

    print(f"Finetune '{finetune_name}' with ID '{finetune_id}' added.")



def show_finetunes():
    url = "https://api.us1.bfl.ai/v1/my_finetunes"
    headers = {
        "Content-Type": "application/json",
        "X-Key": api_key,
    }
    
    response = requests.get(url, headers=headers)
    try:
        response.raise_for_status()
        finetunes_data = response.json()

        with open("finetunes.py", "r") as f:
            content = f.read()
            dict_start = content.find("{")
            dict_end = content.rfind("}") + 1
            dict_content = content[dict_start:dict_end]
            FINETUNES = eval(dict_content)
    
        for finetune_id in finetunes_data['finetunes']:
            found_key = None
            for key, value in FINETUNES.items():
                if value == finetune_id:
                    found_key = key
                    break
            
            if found_key:
                print(f"'{found_key}': '{finetune_id}'")
            else:
                print(f"'None': '{finetune_id}'")
    
    except requests.exceptions.RequestException as e:
        raise requests.exceptions.RequestException(
            f"Finetune listing failed:\n{str(e)}\n{response.content.decode()}"
        )




def delete_finetune(finetune_name):
    with open("finetunes.py", "r") as f:
        content = f.read()
        dict_start = content.find("{")
        dict_end = content.rfind("}") + 1
        dict_content = content[dict_start:dict_end]
        dicts = eval(dict_content) 

    finetune_id = dicts.get(finetune_name)
    if not finetune_id:
        raise ValueError(f"Finetune '{finetune_name}' not found.")

    url = "https://api.us1.bfl.ai/v1/delete_finetune"
    headers = {
        "Content-Type": "application/json",
        "X-Key": api_key,
    }
    payload = {
        "finetune_id": finetune_id,
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  
        print(f"Finetune '{finetune_name}' with ID '{finetune_id}' deleted.")
        
        del dicts[finetune_name]
        
        with open("finetunes.py", "w") as f:
            f.write(f"FINETUNES = {dicts}\n")  
        return response.json()
    
    except requests.exceptions.RequestException as e:
        raise requests.exceptions.RequestException(
            f"Finetune deletion failed:\n{str(e)}\n{response.content.decode()}"
        )

In [ ]:
zip_path = r'zips\researcher.zip'

response = create_finetune(
    zip_path=zip_path,
    iterations=100,
)
response

In [10]:
finetune_id=''
finetune_name=''

add_finetune(
    finetune_id=finetune_id, 
    finetune_name=finetune_name
    )

Finetune 'x1' with ID 'a8a425a8-d7a2-49e4-ab77-f3ac2d978f2f' added.


In [2]:
show_finetunes()

'elvis_frog': '06d461c6-917b-4072-aeed-af3b59a564a9'
'succub': 'e399b503-ff10-46b5-820a-642fd23e028e'
'rhino': 'a29c4469-af23-4891-9bba-20d67c0a8e64'
'researcher': 'f4f7e6f9-9024-44ce-b22d-8bc3ab84bd36'


In [ ]:
from utils import create_images, show_images

succub_prompt = "succubus, standing, wearing green blazer, stretch out a big winner's golden cup with the inscription 'HELLSPIN' in her hands into the viever, enhanced, 4k, high-res, masterpiece, best quality, finely detailed skin, sharp focus, cinematic lighting"
frog_prompt = "elvis, frog, jumping into the viever"

loras = create_images(
    prompt=frog_prompt,
    finetune='elvis_frog',  
    finetune_strength=1.0,
    num_images=1,
    aspect_ratio="1:1",
) 
show_images(loras)

In [ ]:
finetune_name=''

delete_finetune(finetune_name)

Finetune 'succub_v3' with ID '93485517-d3e7-43fb-b384-fcc9542b8d01' deleted.


{'status': 'success',
 'message': 'Finetune successfully deleted',
 'deleted_finetune_id': '93485517-d3e7-43fb-b384-fcc9542b8d01',
 'timestamp': '2025-02-27T23:23:28.819882'}